In [1]:
import numpy as np
import optuna
from optuna.integration import CatBoostPruningCallback
import logging
import sys

import catboost as cb
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "Litmus_ETHUSDT" 
storage_name = "sqlite:///LitmusOptuna.sqlite"

In [3]:
def objective(trial: optuna.Trial) -> float:
    data, target = load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
        "eval_metric": "Accuracy",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
    gbm.fit(
        train_x,
        train_y,
        eval_set=[(valid_x, valid_y)],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(valid_y, pred_labels)

    return accuracy

In [4]:
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5), 
    direction="maximize",
    study_name=study_name, 
    storage=storage_name,
    load_if_exists=True
    )

[I 2022-10-18 21:04:18,781] A new study created in RDB with name: Litmus_ETHUSDT


A new study created in RDB with name: Litmus_ETHUSDT


In [6]:
study.trials_dataframe()["datetime_complete"].max()

""


In [13]:
try:
    latest_trial = study.trials_dataframe()["datetime_complete"].max()
    print(latest_trial)
except:
    print("none")

2022-10-18 21:13:06.641591


In [23]:
latest_trial

Timestamp('2022-10-18 21:13:06.641591')

In [20]:
import pandas as pd
pd.Timedelta("1:00:00")

Timedelta('0 days 01:00:00')

In [22]:
pd.Timestamp.now() - pd.Timedelta("1:00:00")

Timestamp('2022-10-18 20:20:58.645151')

In [25]:
latest_trial < pd.Timestamp.now()

True

In [11]:
study.optimize(objective, n_trials=100, timeout=600)

/var/folders/m4/19hkw84s6rv_8gcz6pd81s9r0035dh/T/ipykernel_50209/3542496604.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
[I 2022-10-18 21:12:35,015] Trial 0 finished with value: 0.958041958041958 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.0245925977605981, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.17411294659197038}. Best is trial 0 with value: 0.958041958041958.


Trial 0 finished with value: 0.958041958041958 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.0245925977605981, 'depth': 11, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.17411294659197038}. Best is trial 0 with value: 0.958041958041958.


/var/folders/m4/19hkw84s6rv_8gcz6pd81s9r0035dh/T/ipykernel_50209/3542496604.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
[I 2022-10-18 21:12:35,328] Trial 1 finished with value: 0.965034965034965 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.029738751214197407, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 8.673309984179019}. Best is trial 1 with value: 0.965034965034965.


Trial 1 finished with value: 0.965034965034965 and parameters: {'objective': 'CrossEntropy', 'colsample_bylevel': 0.029738751214197407, 'depth': 9, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 8.673309984179019}. Best is trial 1 with value: 0.965034965034965.


/var/folders/m4/19hkw84s6rv_8gcz6pd81s9r0035dh/T/ipykernel_50209/3542496604.py:24: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, "Accuracy")
[I 2022-10-18 21:12:36,200] Trial 2 finished with value: 0.951048951048951 and parameters: {'objective': 'Logloss', 'colsample_bylevel': 0.01605945948307907, 'depth': 2, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS'}. Best is trial 1 with value: 0.965034965034965.


...

Trial 97 pruned. Trial was pruned at iteration 5.


[I 2022-10-18 21:13:06,561] Trial 98 pruned. Trial was pruned at iteration 5.


Trial 98 pruned. Trial was pruned at iteration 5.


[I 2022-10-18 21:13:06,643] Trial 99 pruned. Trial was pruned at iteration 5.


Trial 99 pruned. Trial was pruned at iteration 5.


In [10]:
study.best_trial

FrozenTrial(number=72, values=[1.0], datetime_start=datetime.datetime(2022, 10, 18, 20, 28, 2, 308010), datetime_complete=datetime.datetime(2022, 10, 18, 20, 28, 4, 27460), params={'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'colsample_bylevel': 0.08168978100335669, 'depth': 8, 'objective': 'Logloss', 'subsample': 0.9992134234989756}, distributions={'boosting_type': CategoricalDistribution(choices=('Ordered', 'Plain')), 'bootstrap_type': CategoricalDistribution(choices=('Bayesian', 'Bernoulli', 'MVS')), 'colsample_bylevel': FloatDistribution(high=0.1, log=True, low=0.01, step=None), 'depth': IntDistribution(high=12, log=False, low=1, step=1), 'objective': CategoricalDistribution(choices=('Logloss', 'CrossEntropy')), 'subsample': FloatDistribution(high=1.0, log=True, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.916083916083916, 1: 0.951048951048951, 2: 0.9370629370629371, 3: 0.951048951048951, 4: 0.951048951048951, 5: 0.9440559440559441,

In [7]:
study.best_trial

FrozenTrial(number=72, values=[1.0], datetime_start=datetime.datetime(2022, 10, 18, 20, 28, 2, 308010), datetime_complete=datetime.datetime(2022, 10, 18, 20, 28, 4, 27460), params={'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'colsample_bylevel': 0.08168978100335669, 'depth': 8, 'objective': 'Logloss', 'subsample': 0.9992134234989756}, distributions={'boosting_type': CategoricalDistribution(choices=('Ordered', 'Plain')), 'bootstrap_type': CategoricalDistribution(choices=('Bayesian', 'Bernoulli', 'MVS')), 'colsample_bylevel': FloatDistribution(high=0.1, log=True, low=0.01, step=None), 'depth': IntDistribution(high=12, log=False, low=1, step=1), 'objective': CategoricalDistribution(choices=('Logloss', 'CrossEntropy')), 'subsample': FloatDistribution(high=1.0, log=True, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.916083916083916, 1: 0.951048951048951, 2: 0.9370629370629371, 3: 0.951048951048951, 4: 0.951048951048951, 5: 0.9440559440559441,

In [8]:
study.best_params

{'boosting_type': 'Ordered',
 'bootstrap_type': 'Bernoulli',
 'colsample_bylevel': 0.08168978100335669,
 'depth': 8,
 'objective': 'Logloss',
 'subsample': 0.9992134234989756}

In [34]:
gbm = cb.CatBoostClassifier(n_estimators=100, metric_period=20)
gbm

In [35]:
gbm.get_params()

{'metric_period': 20, 'n_estimators': 100}

In [36]:
gbm.set_params(n_estimators=1000)
gbm.get_params()

{'metric_period': 20, 'n_estimators': 1000}